In [10]:
import pandas as pd
import numpy as np
import glob, os
import seaborn as sns
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix,roc_curve,auc, accuracy_score
# bag of words
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


# for LIME import necessary packages
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline
from lime.lime_text import IndexedString,IndexedCharacters
from lime.lime_base import LimeBase
from sklearn.linear_model import Ridge, lars_path
from lime.lime_text import explanation
from functools import partial
import scipy as sp
from sklearn.utils import check_random_state

In [11]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [13]:
# Read the data
df_train=pd.read_csv("/kaggle/input/bangla-fake-news-processed/processed.csv")


#SPLITTING THE TRAINING DATASET INTO TRAINING AND VALIDATION
X_train, X_val, y_train, y_val = train_test_split(df_train["Comments"],df_train["Label"],test_size=0.2, shuffle=True)

In [14]:
# Convert x_train to vector
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_val_vectors_tfidf = tfidf_vectorizer.transform(X_val)
#model
model=RandomForestClassifier(n_estimators = 100, random_state = 10)
model.fit(X_train_vectors_tfidf, y_train) 

#Predict y value for test dataset
y_pred = model.predict(X_val_vectors_tfidf)
y_prob = model.predict_proba(X_val_vectors_tfidf)[:,1]

acc = sklearn.metrics.accuracy_score(y_val, y_pred)
print("Accuracy: {}".format(acc))

print(classification_report(y_val,y_pred))
cm = confusion_matrix(y_val, y_pred)
print('Confusion Matrix:',cm)
f = sns.heatmap(cm, annot=True, fmt='d')

fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [15]:
for i in range(1,20):
    print(X_val.iloc[i])

In [16]:
# Explaining the predictions and important features for predicting the label 1
c = make_pipeline(tfidf_vectorizer, model)
explainer = LimeTextExplainer(class_names=model.classes_)

# classifier_fn is the probability function that takes a string and returns prediction probabilities.
# num_features is the max. number of features we want in the explanation(default is 10).
# labels=(1,) means we want the explanation for the label 1
for i in range(1,50):
    exp = explainer.explain_instance(X_val.iloc[i], c.predict_proba, num_features=5,labels=(1,))
    exp.show_in_notebook()